In [1]:
%load_ext autoreload
%autoreload 2
import pytz
from datetime import datetime, timedelta, date
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from src.inferencesm import load_predictions_from_store

In [2]:



# Define Argentina's timezone (GMT-3)
argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')

# Get the current date and time in Argentina
current_time_in_argentina = datetime.now(argentina_tz).replace(minute=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
# Round down (floor) to the nearest hour by setting minutes, seconds, and microseconds to 0
current_date = pd.to_datetime(current_time_in_argentina)
current_date

Timestamp('2024-10-05 19:00:00')

In [3]:
from src.inferencesm import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

# Create a label encoder object
label_encoder = LabelEncoder()

# Apply label encoding to 'line' and 'station'
features['line'] = label_encoder.fit_transform(features['line'])
features['station'] = label_encoder.fit_transform(features['station'])

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/603286
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2024-09-21 18:00:00+00:00 to 2024-10-05 18:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.54s) 
Index(['station', 'line'], dtype='object')


In [4]:
features

,total_pax_previous_336_hour,total_pax_previous_335_hour,total_pax_previous_334_hour,total_pax_previous_333_hour,total_pax_previous_332_hour,total_pax_previous_331_hour,total_pax_previous_330_hour,total_pax_previous_329_hour,total_pax_previous_328_hour,total_pax_previous_327_hour,...,total_pax_previous_7_hour,total_pax_previous_6_hour,total_pax_previous_5_hour,total_pax_previous_4_hour,total_pax_previous_3_hour,total_pax_previous_2_hour,total_pax_previous_1_hour,hour_of_entry,station,line
18,786.0,469.0,574.0,233.0,193.0,41.0,0.0,0.0,0.0,0.0,...,458.0,1020.0,689.0,885.0,549.0,873.0,598.0,2024-10-05 19:00:00+00:00,0,0
7,452.0,190.0,219.0,75.0,76.0,13.0,0.0,0.0,0.0,0.0,...,173.0,391.0,282.0,410.0,244.0,525.0,329.0,2024-10-05 19:00:00+00:00,1,0
24,1013.0,585.0,773.0,286.0,245.0,36.0,0.0,0.0,0.0,0.0,...,571.0,1146.0,806.0,1073.0,603.0,1138.0,774.0,2024-10-05 19:00:00+00:00,2,1
2,504.0,167.0,279.0,141.0,94.0,15.0,0.0,0.0,0.0,0.0,...,199.0,283.0,197.0,371.0,296.0,514.0,416.0,2024-10-05 19:00:00+00:00,3,2
19,1418.0,699.0,834.0,316.0,442.0,117.0,0.0,0.0,0.0,0.0,...,509.0,1153.0,938.0,1124.0,667.0,1516.0,809.0,2024-10-05 19:00:00+00:00,4,1
27,638.0,346.0,390.0,118.0,128.0,9.0,0.0,0.0,0.0,0.0,...,431.0,742.0,511.0,652.0,326.0,755.0,489.0,2024-10-05 19:00:00+00:00,5,0
35,1780.0,883.0,1342.0,601.0,718.0,112.0,0.0,0.0,0.0,0.0,...,431.0,873.0,679.0,1082.0,634.0,1438.0,904.0,2024-10-05 19:00:00+00:00,6,1
15,2305.0,1049.0,1248.0,615.0,716.0,212.0,0.0,0.0,0.0,0.0,...,475.0,931.0,668.0,1291.0,820.0,1586.0,1155.0,2024-10-05 19:00:00+00:00,7,1
0,650.0,324.0,434.0,172.0,170.0,28.0,0.0,0.0,0.0,0.0,...,298.0,631.0,427.0,734.0,393.0,822.0,535.0,2024-10-05 19:00:00+00:00,8,0
1,1143.0,507.0,705.0,232.0,231.0,78.0,0.0,0.0,0.0,0.0,...,544.0,1055.0,724.0,1115.0,751.0,1401.0,964.0,2024-10-05 19:00:00+00:00,9,0


In [5]:
from src.model_registry import get_latest_model_from_registry
from src.inferencesm import get_model_predictions

model = get_latest_model_from_registry(model_name='subwayBA_passenger_flow_updt', status= 'Production')
predictions = get_model_predictions(model, features)

2024-10-05 19:23:17,608 WARNING: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html

2024-10-05 19:23:17,863 WARNING: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('ruamel')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages

2024-10-05 19:23:19,588 WARNING: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x

2024-10-05 19:23:19,607 WARNING: DeprecationWarning: Using 'method_whitelist' with Retry is deprecated and will be removed in v2.0. Use 'allowed_methods' instead



COMET WARNING: This method has been deprecated, we recommend using the api.get_model(...) method to get the Model Object and then using model.download(...)
COMET INFO: Downloading registry model 'subwayBA_passenger_flow_updt', version '1.1.0', stage None from workspace 'simonmontt'...
COMET INFO: Unzipping model to 'C:\\Users\\PC\\Documents\\Proyect\\passenger_flow_predictor\\models' ...
COMET INFO: done!


2024-10-05 19:23:23,507 WARNING: DeprecationWarning: CountryHoliday is deprecated, use country_holidays instead.

2024-10-05 19:23:23,757 WARNING: InconsistentVersionWarning: Trying to unpickle estimator FunctionTransformer from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

2024-10-05 19:23:23,796 WARNING: InconsistentVersionWarning: Trying to unpickle estimator MultiOutputRegressor from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

2024-10-05 19:23:23,798 WARNING: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.2.2 when using version 1.5.2. This might lead to breaking code 

In [6]:
predictions['hour_of_entry'] = current_date
predictions

,total_pax_next_1_hour,total_pax_next_2_hour,total_pax_next_3_hour,station,line,hour_of_entry
0,987.0,453.0,641.0,0,0,2024-10-05 19:00:00
1,548.0,272.0,235.0,1,0,2024-10-05 19:00:00
2,1188.0,591.0,826.0,2,1,2024-10-05 19:00:00
3,607.0,305.0,311.0,3,2,2024-10-05 19:00:00
4,1641.0,771.0,1018.0,4,1,2024-10-05 19:00:00
5,765.0,311.0,311.0,5,0,2024-10-05 19:00:00
6,1604.0,791.0,1284.0,6,1,2024-10-05 19:00:00
7,2162.0,918.0,1119.0,7,1,2024-10-05 19:00:00
8,788.0,342.0,424.0,8,0,2024-10-05 19:00:00
9,1737.0,1280.0,1039.0,9,0,2024-10-05 19:00:00


In [7]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generated for the next 3 hours by our production model",
    primary_key = ['station', 'line', 'hour_of_entry'],
    event_time='hour_of_entry',
)

Connected. Call `.close()` to terminate connection gracefully.


In [8]:
predictions['station'] = predictions['station'].astype('int32')
predictions['line'] = predictions['line'].astype('int32')
print(predictions.dtypes)
feature_group.insert(predictions, write_options={"wait_for_job": False})

total_pax_next_1_hour           float64
total_pax_next_2_hour           float64
total_pax_next_3_hour           float64
station                           int32
line                              int32
hour_of_entry            datetime64[ns]
dtype: object


Uploading Dataframe: 0.00% |          | Rows 0/44 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group__1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/603286/jobs/named/model_predictions_feature_group__1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2001e2fe8b0>, None)

In [13]:
from src.inferencesm import load_predictions_from_store
current_time = datetime.now().replace(minute=0, second=0, microsecond=0)
#from_hour_of_entry = current_time  
to_hour_of_entry = current_time -timedelta(hours=2)
load_predictions_from_store(to_hour_of_entry)

Connected. Call `.close()` to terminate connection gracefully.
Feature view model_predictions_feature_view_ already exists. Skipping creation. Error: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/603286/featurestores/599109/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270179,"usrMsg":"Feature view: model_predictions_feature_view_, version: 1","errorMsg":"The provided feature view name and version already exists"}', error code: 270179, error msg: The provided feature view name and version already exists, user msg: Feature view: model_predictions_feature_view_, version: 1
Fetching predictions between 2024-10-05 17:00:00 and 2024-10-05 20:00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.58s) 


,total_pax_next_1_hour,total_pax_next_2_hour,total_pax_next_3_hour,station,line,hour_of_entry
24,955.0,603.0,1000.0,0,0,2024-10-05 17:00:00+00:00
26,524.0,381.0,497.0,1,0,2024-10-05 17:00:00+00:00
35,1185.0,754.0,1158.0,2,1,2024-10-05 17:00:00+00:00
7,560.0,534.0,576.0,3,2,2024-10-05 17:00:00+00:00
37,1555.0,898.0,1540.0,4,1,2024-10-05 17:00:00+00:00
40,685.0,476.0,747.0,5,0,2024-10-05 17:00:00+00:00
9,1417.0,976.0,1619.0,6,1,2024-10-05 17:00:00+00:00
8,1774.0,1262.0,2103.0,7,1,2024-10-05 17:00:00+00:00
36,796.0,562.0,779.0,8,0,2024-10-05 17:00:00+00:00
33,1471.0,991.0,1602.0,9,0,2024-10-05 17:00:00+00:00
